In [ ]:
import pandas as pd

from algos import MySVD, MyPLSA, LDAAdapter
from splitter import split
from utils import Transformer

In [ ]:
random_state = 50

In [ ]:
df = pd.read_csv('data/data.csv')

In [ ]:
df_train, df_test = split(df, test_size=0.5, random_state=random_state)

In [ ]:
# SVD
df_neg = create_negative_examples(df=df, df_train=df_train,
                                  num_neg_exs=df_train.shape[0], random_state=self.random_state)
svd = MySVD(num_topics=10)

svd.fit(df_train, df_neg)

# transform df_test
svd_group_encoder_dict = dict(zip(svd.group_encoder.classes_, list(range(len(svd.group_encoder.classes_)))))
encoded_corpus_test = df_test.groupby("user").agg(lambda x: [svd_group_encoder_dict[item] for item in x]).reset_index()

encoded_corpus_test['recs'] = svd.predict(encoded_corpus_test.user.values)
print(mean_average_precision(encoded_corpus_test.group.values, encoded_corpus_test.recs.values))

In [ ]:
# PLSA
transformer = Transformer()

corpus_train = transformer.transform(df_train)
corpus_test = transformer.transform(df_test)


est = MyPLSA(num_topics=10)

num_groups = df.group.nunique()
est.fit(corpus_train, num_groups)

corpus_test['recs'] = est.predict(corpus_test.user.values)

corpus_test['group'] = corpus_test.group.apply(lambda x: [item[0] for item in x])
print(mean_average_precision(corpus_test.group.values, corpus_test.recs.values))

In [ ]:
# LDA
transformer = Transformer()

corpus_train = transformer.transform(df_train)
corpus_test = transformer.transform(df_test)


est = LDAAdapter(num_topics=10)

est.fit(corpus_train)

corpus_test['recs'] = est.predict(corpus_test.user.values)

corpus_test['group'] = corpus_test.group.apply(lambda x: [item[0] for item in x])
print(mean_average_precision(corpus_test.group.values, corpus_test.recs.values))